# Using Postgres to Select Features

In [ ]:
PG_HOST = 'localhost'
PG_PORT = 32780
DATA_FILE = 'data/big_author_data.p'

The objective is to predict those that tweet again. To achieve this:

 * Take the first tweet of every unique author
 * Split them into those that tweet again and those that don't
 * Split them again into 3 groups

In [ ]:
import pickle

with open(DATA_FILE, 'rb') as handle:
    author_data = pickle.load(handle)

In [ ]:
from collections import defaultdict

docs_by_author = defaultdict(list)

for doc in author_data:
    docs_by_author[doc['author']].append(doc)

for docs in docs_by_author.values():
    docs.sort(key=lambda doc: doc['date'])

In [ ]:
repeat_authors = {
    author: docs
    for author, docs in docs_by_author.items()
    if len(docs) > 1
}
one_shot_authors = {
    author: docs
    for author, docs in docs_by_author.items()
    if len(docs) == 1
}

In [ ]:
print(f'Repeat Authors: {len(repeat_authors)}')
print(f'One Shot Authors: {len(one_shot_authors)}')
print(f'{100 * len(retweeters) / len(author_data)}% Repeat Authors')

In [ ]:
def to_sets(repeaters, one_shotters):
    return [
        {
            'repeat authors': r,
            'one shot authors': o
        }
        for r, o in zip(to_threes(repeaters), to_threes(one_shotters))
    ]

def to_threes(data):
    data_items = list(data.items())
    step = len(data) // 3
    return [
        to_dict(collection)
        for collection in [
            data_items[:step], data_items[step:step * 2], data_items[step * 2:]
        ]
    ]

def to_dict(items):
    return {key: value for key, value in items}

investigation, testing, validation = to_sets(repeat_authors, one_shot_authors)

In [ ]:
len(investigation['repeat authors']) + len(testing['repeat authors']) + len(validation['repeat authors'])

Now we need to insert this data into postgres. To do this we need to have a table!

In [ ]:
field_types = {}

for doc_types in [
    {
        field: type(value)
        for field, value in doc.items()
        if value is not None
    }
    for doc in author_data
]:
    for field, field_type in doc_types.items():
        if field not in field_types:
            field_types[field] = field_type
        elif field_types[field] is not field_type:
            raise f'Incompatible field types detected for {field}: {field_types[field]} is not {field_type}'

In [ ]:
field_mapping = {
    str: 'TEXT',
    int: 'BIGINT',
    float: 'NUMERIC',
    bool: 'BOOLEAN'
}
CREATE_TABLE_STATEMENT = f"""
CREATE TABLE documents (
    {
        ", ".join(
            f'"{field}" {field_mapping[field_type]}'
            for field, field_type in field_types.items()
            if field_type in field_mapping
        )
    }
);
"""

In [ ]:
from psycopg2 import connect

with connect(host=PG_HOST, port=PG_PORT, database='postgres', user='postgres') as connection:
    with connection.cursor() as cursor:
        cursor.execute(CREATE_TABLE_STATEMENT)

That is a terrible way to make a table. It did drop every column which never has a value though.

Next to insert the investigation data. This will take a little while to complete.

In [ ]:
keys = [
    field for field, field_type in field_types.items()
    if field_type in field_mapping
]

INSERT_STATEMENT = f'INSERT INTO documents ({", ".join(keys)}) VALUES %s'

In [ ]:
from psycopg2.extras import execute_values

def to_ordered_values(docs_list):
    return [
        [
            doc[key] if key in doc else None
            for key in keys
        ]
        for docs in docs_list
        for doc in docs
    ]

with connect(host=PG_HOST, port=PG_PORT, database='postgres', user='postgres') as connection:
    with connection.cursor() as cursor:
        execute_values(cursor, INSERT_STATEMENT, to_ordered_values(investigation['repeat authors'].values()))
        execute_values(cursor, INSERT_STATEMENT, to_ordered_values(investigation['one shot authors'].values()))

In [ ]:
with connect(host=PG_HOST, port=PG_PORT, database='postgres', user='postgres') as connection:
    with connection.cursor() as cursor:
        cursor.execute('ANALYZE documents;')

Now we can extract the statistics from the table!

In [ ]:
STATISTICS_STATEMENT = """
SELECT
    attname AS column,
    n_distinct,
    most_common_vals,
    most_common_freqs,
    histogram_bounds,
    correlation
FROM pg_stats
WHERE tablename = 'documents';
"""

with connect(host=PG_HOST, port=PG_PORT, database='postgres', user='postgres') as connection:
    with connection.cursor() as cursor:
        cursor.execute(STATISTICS_STATEMENT)
        statistics = cursor.fetchall()

In [ ]:
columns = [
    column
    for column, distinct, *_
    in statistics
    if distinct > 1
]